In [1]:
import pandas as pd
import plotly.express as px
from statsmodels.tsa.seasonal import seasonal_decompose
import logging
import datetime
import time

import dash
from dash import dcc, html
from dash.dependencies import Input, Output

import calendar
import pandas as pd
import plotly.express as px
from pyspark.sql import SparkSession
from pyspark.sql.functions import concat, col
import matplotlib.pyplot as plt
import seaborn as sns
from statsmodels.tsa.seasonal import seasonal_decompose
from ipywidgets import interact, widgets
from matplotlib.dates import MonthLocator, DateFormatter
import geopandas as gpd
from shapely.geometry import Point
from sklearn.preprocessing import StandardScaler
from statsmodels.tsa.holtwinters import ExponentialSmoothing
import geopandas as gpd
from mpl_toolkits.axes_grid1 import make_axes_locatable
#from fbprophet import Prophet
from sklearn.metrics import mean_squared_error
import numpy as np
import plotly.graph_objects as go
from pyspark.sql import functions as F
from pyspark.sql.functions import year, month
from sklearn.metrics.pairwise import euclidean_distances
from pyspark.sql.functions import year, month, sum as sum_
from scipy.spatial.distance import euclidean
from prophet import Prophet
from pyspark.sql.functions import col, countDistinct , desc , count , length
from pyspark.sql.functions import split, col, when
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, udf, dayofmonth, last_day ,to_date, lit, isnull
from pyspark.sql.types import IntegerType
import numpy as np
from sklearn.metrics import r2_score, mean_squared_error
from dateutil.relativedelta import relativedelta
from pyspark.sql.functions import date_format

import schedule
from pyspark.sql.functions import col, to_timestamp, date_format

import pyspark
import pandas as pd
import boto3
import sagemaker
import sagemaker.feature_store.feature_store as fs
import databricks.connect
from dateutil.relativedelta import relativedelta

import pandas as pd
import numpy as np
import plotly.graph_objects as go
from sklearn.metrics import mean_squared_error, mean_absolute_percentage_error, mean_absolute_error
from sklearn.metrics import mean_squared_error, root_mean_squared_error

import pandas as pd
import numpy as np
from prophet import Prophet
from sklearn.metrics import mean_absolute_percentage_error, mean_squared_error
import plotly.graph_objects as go
from itertools import product
import random
import statsmodels.api as sm
import os

import logging

logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
from databricks.connect import DatabricksSession

# Get spark
spark = databricks.connect.DatabricksSession.builder.getOrCreate()

c:\Users\gusta\AppData\Local\pypoetry\Cache\virtualenvs\ltv-novo-aF9Dy8BA-py3.10\lib\site-packages\pydantic\_internal\_fields.py:192: UserWarning: Field name "json" in "MonitoringDatasetFormat" shadows an attribute in parent "Base"
  warnings.warn(


[02/04/25 14:30:53] INFO     Found credentials in shared credentials file: ~/.aws/credentials   ]8;id=356091;file://c:\Users\gusta\AppData\Local\pypoetry\Cache\virtualenvs\ltv-novo-aF9Dy8BA-py3.10\lib\site-packages\botocore\credentials.py\credentials.py]8;;\:]8;id=68964;file://c:\Users\gusta\AppData\Local\pypoetry\Cache\virtualenvs\ltv-novo-aF9Dy8BA-py3.10\lib\site-packages\botocore\credentials.py#1278\1278]8;;\

sagemaker.config INFO - Not applying SDK defaults from location: C:\ProgramData\sagemaker\sagemaker\config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: C:\Users\gusta\AppData\Local\sagemaker\sagemaker\config.yaml


In [2]:

contrato_sdf = spark.table("dev.dw_redeok_model.tb_d_contract")
customer_sdf= spark.table("dev.dw_redeok_model.tb_d_customer")
recebivel_sdf = spark.table("dev.dw_redeok_model.tb_f_accounts_receivable")

In [3]:
from pyspark.sql.functions import col

# Definir o UUID que deseja filtrar
uuid_alvo = "9a2dde7c-04e6-4639-a24d-bfc7c39ce93c"

# Filtrar pelo UUID específico
customer_filtered = customer_sdf.filter(col("uuid") == uuid_alvo)

# Mostrar o resultado
customer_filtered.show()


+--------------------+--------------+-----+--------------------+------------------+--------------------+------------------+--------------------+
|                uuid|      cnpj_cpf|pj_pf|  company_civil_name|data_source_insert|          created_at|data_source_update|          updated_at|
+--------------------+--------------+-----+--------------------+------------------+--------------------+------------------+--------------------+
|9a2dde7c-04e6-463...|23236392000198|   PJ|REDE OK SERVICOS ...|            signet|2024-02-21 13:50:...|                  |2024-02-21 13:50:...|
+--------------------+--------------+-----+--------------------+------------------+--------------------+------------------+--------------------+



In [3]:


recebivel_sdf = recebivel_sdf.orderBy(col("uuid_contract"), col("issue_date"))
recebivel_sdf.show()
recebivel_sdf.count()

+--------------------+--------------------+--------------------+-------------------+----------+----------+--------------------------+------------+-----------+-------+-----------------+-------------------+--------------------+--------------------+------------------+--------------------+------------------+--------------------+
|                uuid|       uuid_contract|        uuid_invoice|       invoice_type|issue_date|  due_date|accounts_receivable_amount|payment_date|paid_amount| status|cancellation_date|cancellation_reason|         observation|      payment_method|data_source_insert|          created_at|data_source_update|           update_at|
+--------------------+--------------------+--------------------+-------------------+----------+----------+--------------------------+------------+-----------+-------+-----------------+-------------------+--------------------+--------------------+------------------+--------------------+------------------+--------------------+
|22949c16-0570-406.

1783013

In [11]:
from pyspark.sql.functions import col, min

from pyspark.sql.functions import min

# Agrupar e calcular a menor data de emissão
recebivel_sdf_z = recebivel_sdf.groupBy("uuid_contract").agg(
    min("issue_date").alias("min_issue_date")
)

# Ordenar pelo menor min_issue_date
recebivel_sdf_z = recebivel_sdf_z.orderBy("min_issue_date")

# Exibir os resultados
recebivel_sdf_z.show(10000)
recebivel_sdf_z.count()



+--------------------+--------------+
|       uuid_contract|min_issue_date|
+--------------------+--------------+
|c785f2d2-3519-468...|    2006-03-01|
|e50352a7-8c60-44d...|    2006-03-01|
|bb9410c3-18f5-403...|    2006-03-01|
|6c8ec998-9b23-4b9...|    2006-03-01|
|90c5649f-d134-432...|    2006-03-01|
|39cbf880-e0ff-48f...|    2006-03-01|
|b351d1a1-90a8-447...|    2006-03-01|
|f2802af9-230c-4a2...|    2006-03-01|
|9b29f107-3b55-4db...|    2006-03-01|
|572646dc-a8d1-42a...|    2006-03-01|
|07d33ef9-46e3-484...|    2006-03-01|
|5623f525-ecf8-426...|    2006-03-01|
|8789fa8f-c3c1-431...|    2006-03-01|
|aaab1a17-3675-48f...|    2006-03-01|
|390313a3-dcff-4bd...|    2006-03-01|
|36e9a584-c609-467...|    2006-03-01|
|84a1d9c3-2fb5-4c2...|    2006-03-01|
|e36b89a5-eaf9-456...|    2006-03-01|
|981902e2-0d8b-491...|    2006-03-01|
|d641e466-e01e-487...|    2006-03-01|
|2eb64855-f5c2-4f2...|    2006-03-01|
|c3178efd-6a3d-4f8...|    2006-03-01|
|b0b91df5-5db8-44e...|    2006-03-01|
|a2d64f98-ac

57370

In [5]:
contrato_sdf.show(truncate=False)
contrato_sdf.count()

+------------------------------------+------------------------------------+------------------------------------+------------------------------------+------------------------------------+--------+-------------------+-------------------+-------------+-----------------+----------+-------------+-------+-----------------+-------------------------+------------------+-----------------------+------------------+--------------------------+---------+-----------+------------------------------------+
|uuid                                |uuid_customer                       |uuid_partner                        |uuid_wallet                         |uuid_business_model                 |status  |start_date         |end_date           |status_reason|consumption_value|access_fee|financial_fee|due_day|payment_frequency|max_ticket_value_in_month|data_source_insert|created_at             |data_source_update|updated_at                |uuid_user|entity_type|uuid_partner_contract               |
+-------------

61168

In [12]:
# Agrupar por uuid_customer, contar e filtrar os que têm mais de um contrato
clientes_multiplos_contratos = contrato_sdf.groupBy("uuid_customer").count() \
    .filter(col("count") > 1) \
    .orderBy(col("count"), ascending=False)

# Exibir o resultado
clientes_multiplos_contratos.show(40, truncate=False)

total_clientes_multiplos_contratos = clientes_multiplos_contratos.count()
print(f"Total de clientes com mais de um contrato: {total_clientes_multiplos_contratos}")

+------------------------------------+-----+
|uuid_customer                       |count|
+------------------------------------+-----+
|9a2dde7c-04e6-4639-a24d-bfc7c39ce93c|275  |
|83a93536-475f-49dc-96dd-2d7081eb9b0d|84   |
|f41ebd28-9111-4fae-8dbf-51e3a43a61ce|19   |
|b034bd72-70bd-4251-9a8d-b08656b25508|17   |
|935c08da-bb9e-4cc9-8536-65492aad220e|14   |
|85c3d580-f467-4499-a818-d852791e9400|11   |
|c765a82a-072c-41bb-a253-66998011162d|10   |
|337bce7c-088c-457f-a2fe-570a6c9d2792|9    |
|08dd6f9a-5f31-46b7-a061-09bad0f5bade|9    |
|65255387-2b06-4cb6-91ec-7cc20262067e|8    |
|41a0ee2b-dd6c-43b7-8430-5adf1fdc523d|8    |
|014e0784-d1ea-40af-8d38-96493d33e779|8    |
|b93ce734-5c69-4fd5-90c2-368a9d08cbf5|7    |
|c0f90c1e-d032-4d20-8f91-d3755670cc87|7    |
|5a151ad9-d864-4ed3-b006-251a79094a62|6    |
|c6b515cb-87d1-4153-9d32-aa1d1f092285|6    |
|ef141ca9-5bb1-4ae8-81f2-3f03005b1700|6    |
|dda0965c-ba5a-4ed5-af98-d3c4e610b080|6    |
|97a0a465-7959-40c1-8784-ea1fc692ddca|5    |
|651b4178-

In [14]:
customer_sdf.show()
customer_sdf.count()


+--------------------+--------------+-----+--------------------+------------------+--------------------+------------------+--------------------+
|                uuid|      cnpj_cpf|pj_pf|  company_civil_name|data_source_insert|          created_at|data_source_update|          updated_at|
+--------------------+--------------+-----+--------------------+------------------+--------------------+------------------+--------------------+
|fd614f09-8be0-407...|39642197000106|   PJ|Solar Matriz Come...|            zordon|2024-11-19 17:00:...|                  |2024-11-19 17:00:...|
|5c326e67-f171-447...|18715285000102|   PJ|Eletrica Lider Co...|            zordon|2024-11-19 17:00:...|                  |2024-11-19 17:00:...|
|20aec9e5-7fbc-4c2...|17926427000118|   PJ|Martins e Veronez...|            zordon|2024-11-19 17:00:...|                  |2024-11-19 17:00:...|
|34a0ad4c-11e9-41a...|23205526000103|   PJ|23.205.526 Jose O...|            zordon|2024-10-28 19:20:...|                  |2024-10

57110

In [20]:
# Contar quantas chaves únicas existem em cada tabela
print("UUIDs Contratos únicos em recebivel_sdf:", recebivel_sdf.select("uuid_contract").distinct().count())

print("UUIDs únicos em recebivel_sdf:", recebivel_sdf.select("uuid").distinct().count())

UUIDs Contratos únicos em recebivel_sdf: 57370
UUIDs únicos em recebivel_sdf: 1783013


In [21]:
# Contar quantas chaves únicas existem em cada tabela

print("UUIDs únicos em contrato_sdf:", contrato_sdf.select("uuid").distinct().count())

print("UUIDs únicos (clientes) em contrato_sdf:", contrato_sdf.select("uuid_customer").distinct().count())

UUIDs únicos em contrato_sdf: 61168
UUIDs únicos (clientes) em contrato_sdf: 57213


In [22]:
# Contar quantas chaves únicas existem em cada tabela
#print("UUIDs Customers únicos em contrato_sdf:", contrato_sdf.select("uuid_customer").distinct().count())

print("UUIDs únicos em customer_sdf:", customer_sdf.select("uuid").distinct().count())
print("CNPJs únicos em customer_sdf:", customer_sdf.select("cnpj_cpf").distinct().count())


UUIDs únicos em customer_sdf: 57110
CNPJs únicos em customer_sdf: 57104


- faço um left ou inner?

In [8]:
from pyspark.sql.functions import col

# Realizando o JOIN
resultado_sdf = recebivel_sdf.join(contrato_sdf, recebivel_sdf.uuid_contract == contrato_sdf.uuid, "left")

# Ordenando por uuid_contract e issue_date (ambos em ordem crescente)
resultado_sdf = resultado_sdf.orderBy(col("uuid_contract"), col("issue_date"))

# Exibir o resultado
resultado_sdf.show()

resultado_sdf.count()

+--------------------+--------------------+--------------------+-------------------+----------+----------+--------------------------+------------+-----------+-------+-----------------+-------------------+--------------------+--------------------+------------------+--------------------+------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+------+-------------------+--------+-------------+-----------------+----------+-------------+-------+-----------------+-------------------------+------------------+--------------------+------------------+--------------------+---------+-----------+---------------------+
|                uuid|       uuid_contract|        uuid_invoice|       invoice_type|issue_date|  due_date|accounts_receivable_amount|payment_date|paid_amount| status|cancellation_date|cancellation_reason|         observation|      payment_method|data_source_insert|          created_at|data_source_u

1783013